In [ ]:
import pandas as pd
import numpy as np

import sklearn.metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import keras
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2025-01-04 06:59:14.494546: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-04 06:59:14.528254: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
def get_data(gyro='EC2', mode=-1):
    # Define your input and output columns
    input_columns = [
        'pol_ang', 'tor_ang', 'Bt_center', 'Ip', 'R0', 'Z0', 'aminor', 'betan', 
        'elong', 'li', 'volume', 
        'ne_val_0', 'ne_val_10', 'ne_val_20'
    ]

    output_columns = ['rho_pol', 'R', 'Z', 'CD_eta']

    # Load the data
    df_initial = pd.read_csv("TORBEAM_all_df.csv")  # replace with your CSV file path

    # Narrow down to a specific gyro and mode
    temp = df_initial[df_initial['gyro'] == gyro]
    temp = temp[temp['mode'] == mode]
    # Remove rho_pol of -1
    temp = temp[temp['rho_pol'] != -1]
    df = temp.dropna()

    # Extract input features and output targets
    X = df[input_columns].values
    y = df[output_columns].values

    # Optional: Perform a train-validation-test split
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test   = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    # Optional: Normalize features (recommended for neural networks)
    scaler_X = StandardScaler()
    X_train = scaler_X.fit_transform(X_train)
    X_val = scaler_X.transform(X_val)
    X_test = scaler_X.transform(X_test)

    # If desired, you can also scale the outputs, especially if their ranges differ significantly.
    scaler_y = StandardScaler()
    y_train = scaler_y.fit_transform(y_train)
    y_val = scaler_y.transform(y_val)
    y_test = scaler_y.transform(y_test)

    return X_train, X_val, X_test, y_train, y_val, y_test, scaler_X, scaler_y

In [5]:
output_names = ['rho_pol', 'R', 'Z', 'CD_eta']
ind = 0
full_df = None

for gyro in ['EC2', 'EC4', 'EC5']:
    for mode in [-1, 1]:
        X_train, X_val, X_test, y_train, y_val, y_test, _, _ = get_data(gyro=gyro, mode=mode)

        for num_units in [60,80,100,120]:
            if mode == -1:
                mode_name = 'xmode'
                plt_name = 'X-Mode'
            else:
                mode_name = 'omode'
                plt_name = 'O-Mode'
            model = keras.saving.load_model(f'models/{gyro}_{mode_name}_{num_units}.keras')
            y_pred = model.predict(X_test)
            with open(f'models/{gyro}_{mode_name}_{num_units}.pkl', 'rb') as f:
                training_info = pickle.load(f)

            accuracy = training_info['accuracy']
            history = training_info['history']
            scaler_X = training_info['scaler_X']
            scaler_y = training_info['scaler_y']

            y_test_plot = scaler_y.inverse_transform(y_test)
            y_pred_plot = scaler_y.inverse_transform(y_pred)


            d = {}
            d['gyro'] = gyro
            d['mode'] = mode
            d['num_units'] = num_units
            for i, output_name in enumerate(output_names):
                r2_score = sklearn.metrics.r2_score(y_test_plot[:, i], y_pred_plot[:, i])
                d[output_name] = r2_score

            df = pd.DataFrame(d, index=[ind])
            if full_df is None:
                full_df = df
            else:
                full_df = pd.concat([full_df, df])


2056/2056 [==============================] - 1s 604us/step


In [17]:
full_df[np.logical_and(full_df['gyro'] == 'EC2', full_df['mode'] == -1)]

,gyro,mode,num_units,rho_pol,R,Z,CD_eta
0,EC2,-1,60,0.995904,0.993927,0.998411,0.994615
0,EC2,-1,80,0.996461,0.994624,0.998486,0.996158
0,EC2,-1,100,0.997170,0.994897,0.998918,0.997012
0,EC2,-1,120,0.997328,0.994874,0.998863,0.997165


### Look at normalizationns

In [3]:
with open('models/EC2_xmode_60.pkl', 'rb') as f:
    training_info = pickle.load(f)

In [4]:
training_info.keys()

dict_keys(['accuracy', 'history', 'scaler_X', 'scaler_y'])

In [43]:
ind = 0
for i in range(ind,ind+1):
    for gyro in ['EC2', 'EC4', 'EC5']:
        with open(f'models/{gyro}_xmode_60.pkl', 'rb') as f:
            training_info = pickle.load(f)

        print(f'{gyro} {training_info["scaler_X"].mean_[i]}')

    for gyro in ['EC2', 'EC4', 'EC5']:
        with open(f'models/{gyro}_xmode_60.pkl', 'rb') as f:
            training_info = pickle.load(f)
        print(f'{gyro} {training_info["scaler_X"].scale_[i]}')
    # (training_info['scaler_X'].mean_[2:])
    # (training_info['scaler_X'].scale_[2:])
    #print(training_info['scaler_y'].mean_)
    # print(training_info['scaler_y'].scale_)

EC2 0.31741317934357893
EC4 -0.34219577294018877
EC5 -0.33584843979968826
EC2 0.1037795888951573
EC4 0.11791940037597994
EC5 0.11580169210313876


In [40]:
model = keras.saving.load_model(f'models/EC2_xmode_60.keras')

In [ ]:
trial_input = np.array([0.304999, 
                        0.75498, 
                        0.781511, 
                        0.099258,
                        -3.818018, 
                        0.520651, 
                        -1.254566,
                        -0.447334, 
                        -4.406280, 
                        -1.818545, 
                        -3.939891, 
                        0.035734, 
                        0.430360, 
                        0.52868
                        ])
trial_input = trial_input.reshape(1, -1)

output = model.predict(trial_input)
with open(f'models/{gyro}_xmode_60.pkl', 'rb') as f:
    training_info = pickle.load(f)

print(training_info['scaler_y'].inverse_transform(output))

AttributeError: module 'keras' has no attribute 'tensor'

In [98]:
with open(f'models/EC5_omode_60.pkl', 'rb') as f:
    training_info = pickle.load(f)

print(training_info['scaler_y'].mean_)
print(training_info['scaler_y'].scale_)

[ 3.39217331e-01  1.70878825e+00 -6.74200280e-02 -2.70605993e-04]
[0.11401295 0.04625591 0.12267106 0.00925621]


## Check PCS calculation

### EC2

In [ ]:
model = keras.saving.load_model(f'models/EC2_xmode_60.keras')
with open(f'models/EC2_xmode_60.pkl', 'rb') as f:
    training_info = pickle.load(f)

raw_input = np.array([0.261804, 
                        0.0, 
                        -1.791502, 
                        -515634.718,
                        1.77, 
                        0.0, 
                        0.459118*100,
                        0.950630, 
                        1.450798, 
                        0.849321, 
                        10.160793*1e6, 
                        3.598558, 
                        3.115731, 
                        0.209706
                        ])

raw_input = raw_input.reshape(1, -1)

trial_input = training_info['scaler_X'].transform(raw_input)
print(trial_input)

output = model.predict(trial_input)
print(output[0,1:3])
print(training_info['scaler_y'].inverse_transform(output)[0,1:3])

[[-0.53583927 -0.00451886  0.05483941 -0.04150851 -1.97538548  0.49219788
  -0.37763258 -0.41612103 -4.1282275  -1.67131156 -3.61911192  0.01422088
   0.47756198  0.58284796]]
1/1 [==============================] - 0s 42ms/step
[-0.58367443  0.5007925 ]
[1.7221699  0.17023885]


In [11]:
print(training_info['scaler_X'].mean_[10])
print(training_info['scaler_X'].scale_[10])

12777734.913684051
723089.522511209


In [42]:
inputs = [-5.89540962e-01,-5.31455719e-01,+4.50318004e-01,-1.09321609e+00,-1.48777925e+00,
-1.53913852e+00,-1.17093095e+00,+6.44183951e-01,+1.04089078e+00,+1.42375586e+00,
-9.33865478e-01,+1.88388224e+00,+7.64007621e-01,-7.92732182e-03]

raw_input = np.array(inputs).reshape(1, -1)

model = keras.saving.load_model(f'models/EC2_xmode_60.keras')

output = model.predict(raw_input)
print(output)

1/1 [==============================] - 0s 44ms/step
[[ 1.7654405  -0.51966643  1.0948038   0.7874182 ]]


In [43]:
[+5.53693235e-01,-2.96740413e-01,+5.07193923e-01,+8.95579234e-02]

[0.553693235, -0.296740413, 0.507193923, 0.0895579234]

In [44]:
model.layers[0].get_weights()[0][0,0]

0.020018024

In [41]:
model.layers[0].get_weights()[0]

array([[ 2.00180244e-02,  2.25528162e-02,  6.89424992e-01,
        -9.32391081e-03,  1.36226714e-02, -2.27822224e-03,
         3.52142123e-03,  2.21876390e-02,  7.84411654e-02,
         1.01493308e-02,  7.84593225e-02, -1.67687722e-02,
        -1.17378682e-02, -6.32737298e-03,  6.84254229e-01,
         2.81457826e-02, -3.24052584e-04, -2.81280398e-01,
         1.67875551e-02,  5.01174806e-03,  2.26888992e-02,
        -2.69525591e-03,  3.37144703e-01, -7.26191550e-02,
        -3.07646573e-01,  5.07269084e-01,  3.19633546e-04,
        -1.83256436e-02, -2.28847891e-01,  4.82691191e-02,
         2.91315943e-01, -6.57258136e-03,  3.55967171e-02,
         2.17133388e-02,  1.57921836e-01,  3.00484687e-01,
        -2.18333807e-04, -1.91089232e-02, -4.07347409e-03,
         3.73413190e-02, -5.87272227e-01,  7.31675863e-01,
        -7.20458567e-01, -2.65119411e-03,  1.87123148e-03,
        -9.72618997e-01,  5.14735188e-03, -1.38757611e-02,
        -1.61703415e-02,  1.10167670e+00,  7.72577524e-0

### EC4